In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import astropy.coordinates as coord
from astropy.time import Time
import astropy.units as u
import time

In [ ]:
def convert(t):
    return time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S'))

def convert_deg(d):
    deg=d.split("d")[0]
    m=d.split("d")[1].split("m")[0]
    s=d.split("m")[1].split("s")[0]
    deg=float(deg)+float(m)/60+float(s)/3600
    return deg

def time_to_float(x):
    H=float(x.split(":")[0])
    M=float(x.split(":")[1])
    return H+M/60.0
def ZM(kot):
    return 1.0/(np.cos(kot*np.pi/180.0)+0.50572*np.power(96.07995-kot,-1.6364))

def Rp(kot1,kot2):
    return np.power(0.7,np.power(ZM(kot1),0.678))*np.cos(kot2*np.pi/180.0)

def d2r(x):
    return np.deg2rad(x)

def f_vzhod(alfa,beta):
    s1=np.sin(d2r(alfa))*np.cos(d2r(beta))
    if (s1<0 or beta<0):
        s1=0
    return (s1*35*5.9)#/np.sin(d2r(alfa))

def f_jug(alfa,beta):
    s1=np.cos(d2r(alfa-180))*np.cos(d2r(beta))
    if (s1<0 or beta<0):
        s1=0
    return (s1*12*5.9)#/np.sin(d2r(alfa))

def f_zahod(alfa,beta):
    s1=np.sin(d2r(alfa)+180)*np.cos(d2r(beta))
    if (s1<0 or beta<0):
        s1=0
    return (s1*35*5.9)#/np.sin(d2r(alfa))

def f_gori(alfa,beta):
    s1=np.cos(d2r(beta-90))
    if s1<0:
        s1=0
    return (s1*35*12)#/np.sin(d2r(alfa))

def f(alfa,beta):
    return [f_vzhod(alfa,beta),f_jug(alfa,beta),f_zahod(alfa,beta),f_gori(alfa,beta)]

def simulate_a_year(T,lam,S,h,nu,alb,Sr,key,regulate=False):
    Tn=T
    eps=5.67*np.power(10,-8.0)
    jj={}
    jj["A"]=[]
    jj["B"]=[]
    jj["C"]=[]
    jj["D"]=[]
    jj["Tot"]=[]
    jj["Q"]=[]
    jj["Tn"]=[]
    jj["To"]=[]
    for i,ii in df_all.iterrows():
        ja=nu*ii[key]
        if str(ii["Id"])=="nan":
            ja=0
        jb=0.0#alb*eps*(np.power(ii["T"]+273,4.0)-np.power(Tn+273,4.0))*Sr
        jc=lam*(ii["T"]-Tn)*Sr
        jj["A"].append(ja)
        jj["B"].append(jb)
        jj["C"].append(jc)
        jj["Tot"].append(jj["A"][-1]+jj["B"][-1]+jj["C"][-1])
        yy=np.array(jj["A"])
        yy1=np.array(jj["B"])
        yy2=np.array(jj["C"])
        jj["Q"].append(np.sum(yy)+np.sum(yy1)+np.sum(yy2))
        if regulate==False:
            Tn=Tn+jj["Tot"][-1]*3600/(S*h*1.29*1005)
        else:
            Tn=Tn+jj["Tot"][-1]*3600/(S*h*1.29*1005)
            Q=S*h*1.29*1005*(T-Tn)
            print(Tn,Q,jj["Tot"],jj["A"],jj["C"])
            Tn=Tn+Q/(S*h*1.29*1005)
            jj["D"].append(Q)
        jj["Tn"].append(Tn)
        jj["To"].append(ii["T"])
    if len(jj["D"])<1:
        jj["D"]=[0 for i in range(len(jj["A"]))]
    return jj

In [ ]:
df_Vreme=pd.read_csv("Vreme_urni.csv")
display(df_Vreme)

df_1h=pd.read_csv("df_1h.csv")
display(df_1h.loc[10:20,:])

In [ ]:
try:
    df_1h=df_1h.drop(columns=["T"])
    display(df_1h)
except:
    pass
df_all=pd.merge(df_1h, df_Vreme, on = "date", how = "left")
for i,ii in df_all.iterrows():
    df_all.loc[i,"T"]=df_Vreme.loc[i,"T"]
    df_all.loc[i,"J"]=df_Vreme.loc[i,"J"]
    df_all.loc[i,"Jd"]=df_Vreme.loc[i,"J"]*df_all.loc[i,"Sr"]/1000
print(df_all["T"].min())

In [ ]:
lam=0
a=24.0
b=35.0
h=5.9
S=a*b
Sr=2*a*h+2*b*h+S
V=a*b*h
key=["Jd","Id"]

In [ ]:
T=20
k=0
plt.figure(figsize=(14,10),dpi=300)

lam=7.5
ic=0
alb_p=[0.00001,0.0001,0.001,0.01]

TT=0.9

print("Subplot 1")
plt.subplot(221)
r=simulate_a_year(T,lam,S,h,TT,alb_p[ic],Sr,key[k],regulate=False)
for T in range(10,26,5):
    print(T,"Done")
    plt.plot([yy-T for yy in r["To"]],label="$T$ = %.0f°C"%T,lw=0.2,alpha=1)
#plt.plot(r["To"],label="Okolica",lw=1,alpha=0.2,color="black")
#plt.xlim(0,364*24)
plt.xticks([i for i in range(1,364*24,60*24)],["Jan","Mar","Maj","Jul","Sep","Nov","jan"])
plt.xlim(0,350*24)
#plt.ylim(-15,75)
plt.ylabel("$\Delta $ [°C]")
plt.legend()



plt.subplot(222)
print("Subplot 1")
for T in range(20,21,5):
    r=simulate_a_year(T,lam,S,h,TT,alb_p[ic],Sr,key[k],regulate=False)
    print(T,"Done")
    jj=[np.sum(r["A"][0:kk])/1000 for kk in range(len(r["A"]))]
    plt.plot(jj,label="$T$ = %.0f°C"%T,lw=1,alpha=1)
#plt.plot(r["To"],label="Okolica",lw=1,alpha=0.2,color="black")
#plt.xlim(0,364*24)
plt.xticks([i for i in range(1,364*24,60*24)],["Jan","Mar","Maj","Jul","Sep","Nov","jan"])
plt.xlim(0,350*24)
#plt.ylim(-15,75)
plt.ylabel("$Q_{A}$ [kWh]")

plt.subplot(223)
print("Subplot 2")
for T in range(10,28,5):
    r=simulate_a_year(T,lam,S,h,TT,alb_p[ic],Sr,key[k],regulate=False)
    print(T,"Done")
    jj=[np.sum(r["B"][0:kk])/1000 for kk in range(len(r["B"]))]
    plt.plot(jj,label="$T$ = %.0f°C"%T,lw=1)
plt.legend()
#plt.xlim(0,364*24)
plt.xticks([i for i in range(1,364*24,60*24)],["Jan","Mar","Maj","Jul","Sep","Nov","jan"])
plt.xlim(0,350*24)
#plt.ylim(-15,75)
plt.ylabel("$Q_{B}$ [kWh]")

plt.subplot(224)
print("Subplot 3")
for T in range(10,28,5):
    r=simulate_a_year(T,lam,S,h,TT,alb_p[ic],Sr,key[k],regulate=False)
    print(T,"Done")
    jj=[np.sum(r["C"][0:kk])/1000 for kk in range(len(r["C"]))]
    plt.plot(jj,label="$T$ = %.0f°C"%T,lw=1,alpha=1)
#plt.plot(r["To"],label="Okolica",lw=1,alpha=0.2,color="black")
plt.legend()
#plt.xlim(0,364*24)
plt.xticks([i for i in range(1,364*24,60*24)],["Jan","Mar","Maj","Jul","Sep","Nov","jan"])
plt.xlim(0,350*24)
#plt.ylim(-15,75)

plt.ylabel("$Q_{C}$ [kWh]")
plt.show()

In [ ]:
for T in range(20,21,5):
    r=simulate_a_year(T,lam,S,h,TT,0,Sr,key[k],regulate=True)

In [ ]:
print(len(r["A"]))
print(len(r["B"]))
print(len(r["C"]))

plt.plot(r["C"])
plt.plot(r["A"])
plt.show()

plt.plot(np.array(r["D"])/3600)
plt.show()

In [ ]:
plt.plot(r["Tn"])